In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':"1C4bfP-e2Uoiwc0xZYprS7jar3B9Iy6dB"})  
downloaded.GetContentFile('reddit.tsv')       

In [81]:
import spacy

import spacy.cli
spacy.cli.download("en_core_web_lg")
import en_core_web_lg
nlp = en_core_web_lg.load()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [82]:
data = pd.read_csv('reddit.tsv', sep="\t")
 
print(data.shape)

(1013000, 4)


In [0]:
subrs = data['subreddit'].unique().tolist()

In [0]:
# type(subrs[0])

In [0]:
# nomen = 'number'+subrs[0]
# vars()[nomen] = 456
# numbertalesfromtechsupport

In [0]:
# vars()['nomen']

In [0]:
# nomenlist = []
# nomenlist.append(vars()['nomen'])
# nomenlist

In [0]:
# vars()[nomenlist[0]]

In [0]:
# data[data['subreddit'] == subrs[0]].head()

In [90]:
samplist = []

for i in range(len(subrs)):
  nomen = 'datasamp'+str(i)
  vars()[nomen] = data[data['subreddit'] == subrs[i]]
  samplist.append(nomen)

len(samplist)

1013

In [0]:
# vars()[samplist[0]].head()

In [0]:
# vars()[samplist[0]].shape

In [0]:
# test = pd.concat([vars()[samplist[0]], vars()[samplist[1]]], ignore_index=True)

# test.shape

In [0]:
# len(samplist)-1

In [95]:
# CREATE DATA_STRAT
full_starter = vars()[samplist[0]]
data_strat = full_starter.sample(frac=0.04, replace=True, random_state=42)

for i in range(len(samplist)-1):
  full_subr = vars()[samplist[i+1]]
  subr_samp = full_subr.sample(frac=0.04, replace=True, random_state=42)
  data_strat = pd.concat([data_strat, subr_samp], ignore_index=True)

data_strat.shape

(40520, 4)

In [0]:
# checkr = data_strat['subreddit'].unique().tolist()

In [0]:
# checkr == subrs

In [0]:
# data_strat.head(50)

In [0]:
# data_strat.tail(50)

In [100]:
# load data

!pip install PyDrive

In [0]:
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel

In [0]:
# :: Cleaning :: #

import re
def clean(X):

    # Prevent SettingWithCopyWarning
    X = X.copy()
    
    # remove '\\n'
    X['selftext'] = X['selftext'].map(lambda x: re.sub('\\n',' ',str(x)))
    X['subreddit'] = X['subreddit'].map(lambda x: re.sub('\\n',' ',str(x)))
    
    # remove '' 
    X['selftext'] = X['selftext'].map(lambda x: re.sub('<lb>',' ',str(x)))
    X['subreddit'] = X['subreddit'].map(lambda x: re.sub('<lb>',' ',str(x)))
    
    # remove any text starting with User... 
    X['selftext'] = X['selftext'].map(lambda x: re.sub("\[\[User.*",'',str(x)))
    X['subreddit'] = X['subreddit'].map(lambda x: re.sub("\[\[User.*",'',str(x)))
    
    # remove IP addresses or user IDs
    X['selftext'] =X['selftext'].map(lambda x: re.sub("\[\[User.*",'',str(x)))
    X['subreddit'] = X['subreddit'].map(lambda x: re.sub("\[\[User.*",'',str(x)))
    
    #remove http links in the text
    X['selftext'] = X['selftext'].map(lambda x: re.sub("(http://.*?\s)|(http://.*)",'',str(x)))
    X['subreddit'] = X['subreddit'].map(lambda x: re.sub("(http://.*?\s)|(http://.*)",'',str(x)))
    
    X['corpus'] = X['title'] + ' ' + X['selftext']
    

    return X

# train = clean(train)    
# test = clean(test)

In [103]:
data_strat0 = clean(data_strat)

print(data_strat0.shape)
data_strat0.head()

(40520, 5)


,id,subreddit,title,selftext,corpus
0,6k0qhd,talesfromtechsupport,"User reboots device, problem still here","This story happened 4 years ago, while I was s...","User reboots device, problem still here This s..."
1,7zpl52,talesfromtechsupport,Nothing but the finest in safety plans that ma...,"Ok, so the other day I told you about a system...",Nothing but the finest in safety plans that ma...
2,85npi2,talesfromtechsupport,"Sorry, I don't keep any record of temporary pa...",[My previous tale](https://www.reddit.com/r/ta...,"Sorry, I don't keep any record of temporary pa..."
3,7vq5dh,talesfromtechsupport,I didn't want someone to trip on the cable.,First time posting here so I'm going to keep i...,I didn't want someone to trip on the cable. Fi...
4,8huyw0,talesfromtechsupport,The ole' Contrast knob.,So this goes back a few years to when I was an...,The ole' Contrast knob. So this goes back a fe...


In [104]:
data_strat0['corpus'] = data_strat0['title'] + ' ' + data_strat0['selftext']
print(data_strat0.shape)
data_strat0.head()

(40520, 5)


,id,subreddit,title,selftext,corpus
0,6k0qhd,talesfromtechsupport,"User reboots device, problem still here","This story happened 4 years ago, while I was s...","User reboots device, problem still here This s..."
1,7zpl52,talesfromtechsupport,Nothing but the finest in safety plans that ma...,"Ok, so the other day I told you about a system...",Nothing but the finest in safety plans that ma...
2,85npi2,talesfromtechsupport,"Sorry, I don't keep any record of temporary pa...",[My previous tale](https://www.reddit.com/r/ta...,"Sorry, I don't keep any record of temporary pa..."
3,7vq5dh,talesfromtechsupport,I didn't want someone to trip on the cable.,First time posting here so I'm going to keep i...,I didn't want someone to trip on the cable. Fi...
4,8huyw0,talesfromtechsupport,The ole' Contrast knob.,So this goes back a few years to when I was an...,The ole' Contrast knob. So this goes back a fe...


In [0]:
# type(data_strat0['corpus'][0])

In [0]:
# data_strat0['corpus'][0]

In [107]:
train, test = train_test_split(data_strat0, test_size=0.2, stratify=data_strat0["subreddit"])

train.shape, test.shape

((32416, 5), (8104, 5))

In [108]:
X_train = train["corpus"]
X_test = test["corpus"]

y_train = train["subreddit"]
y_test = test["subreddit"]

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(32416,) (8104,)
(32416,) (8104,)


In [0]:
def tokenize(document):
    doc = nlp(document)
    return [token.lemma_.strip().lower() for token in doc 
            if (token.is_stop is False) and (token.is_punct is False)
            and (token.is_space is False)]

In [0]:
tfidf = TfidfVectorizer(
    stop_words='english', sublinear_tf=True, strip_accents='unicode',
    ngram_range=(1,2), max_df=0.97, min_df=3, max_features=1000,
    analyzer='word', tokenizer=tokenize
)

In [111]:
vectorizer = tfidf.fit(X_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [0]:
X_train_vect = vectorizer.transform(X_train)

In [0]:
dtm = pd.DataFrame(X_train_vect.todense(), columns=tfidf.get_feature_names())

In [114]:
dtm.shape

(32416, 1000)

In [0]:
# dtm.to_csv(index=False)

In [116]:
y_train.shape

(32416,)

In [0]:
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [0]:
# grid_params = {
#     'n_neighbors': [2,4,8,12,16],
#     'weights': ['uniform', 'distance'],
#     'metric': ['euclidean','manhattan']
# }


# gs = GridSearchCV(
#     KNeighborsClassifier(), grid_params, verbose=1, cv=5, n_jobs=-1
# )

# gs_results = gs.fit(dtm, y_train)

# print(gs_results.best_score_)
# print(gs_results.best_estimator_)
# print(gs_results.best_params_)

# # y_pred = knc.predict(X_train_vect)
# # print("Accuracy:", metrics.accuracy_score(y_train, y_pred))

In [119]:
knc = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=-1, n_neighbors=16, p=2,
                     weights='distance')
knc.fit(dtm, y_train)
y_pred = knc.predict(dtm)
print("Accuracy:", metrics.accuracy_score(y_train, y_pred))

Accuracy: 0.9999691510365252


In [0]:
X_test_vect = vectorizer.transform(X_test)

In [0]:
test_dtm = pd.DataFrame(X_test_vect.todense(), columns=tfidf.get_feature_names())

In [0]:
test_pred = knc.predict(test_dtm)

In [123]:
print("Accuracy:", metrics.accuracy_score(y_test, test_pred))

Accuracy: 0.175715695952616


In [0]:
test_input = """
Michael Jordan on Isiah Thomas: "Whatever he says now, you know it wasn't his true actions then. 
He's had time to think about it. Or, the reaction of the public, that's kind of changed his 
perspective of it. You can show me anything you want. There's no way you can convince me he wasn't an asshole.
"""

In [129]:
test_user_vect = vectorizer.transform([test_input])
test_user_dtm = pd.DataFrame(test_user_vect.todense(), columns=tfidf.get_feature_names())
knc.predict(test_user_dtm)


array(['survivinginfidelity'], dtype=object)

In [0]:
# !pip install bert-for-tf2

In [0]:
# import tensorflow_hub as hub
# import tensorflow as tf
# import bert
# FullTokenizer = bert.bert_tokenization.FullTokenizer
# from tensorflow.keras.models import Model
# import math

In [0]:
# max_seq_length = 200

In [0]:
# input_word_ids = tf.keras.layers.Input(
#     shape = (max_seq_length,), dtype=tf.int32, name = 'input_word_ids'
# )

# input_mask = tf.keras.layers.Input(
#     shape = (max_seq_length,), dtype=tf.int32, name = 'input_mask'
# )

# segment_ids = tf.keras.layers.Input(
#     shape = (max_seq_length,), dtype=tf.int32, name = 'segment_ids'
# )

# bert_layer = hub.KerasLayer(
#     "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
#     trainable=True
# )

# pooled_output, sequence_output = bert_layer(
#     [input_word_ids, input_mask, segment_ids]
# )

# model = Model(
#     inputs = [input_word_ids, input_mask, segment_ids],
#     outputs = [pooled_output, sequence_output]
# )

In [0]:
# def get_masks(tokens, max_seq_length):
#   if len(tokens) > max_seq_length:
#     raise IndexError("Token length more than max_sequence_length")
#   return [1]*len(tokens) + [0]*(max_seq_length-len(tokens))

In [0]:
# def get_segments(tokens, max_seq_length):
#   if len(tokens) > max_seq_length:
#     raise IndexError("Token length more than max_sequence_length")
#   segments = []
#   current_segment_id = 0
#   for token in tokens:
#     segments.append(current_segment_id)
#     if token == "[SEP]":
#       current_segment_id = 1
#   return segments + [0]*(max_seq_length-len(tokens))

In [0]:
# def get_ids(tokens, tokenizer, max_seq_length):
#   token_ids = tokenizer.convert_tokens_to_ids(tokens)
#   input_ids = token_ids + [0]*(max_seq_length-len(token_ids))
#   return input_ids

In [0]:
# vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
# do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
# tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [0]:
# test_tokens = tokenizer.tokenize('succinct sentence')                 # X_train[0][:200])
# test_tokens = ["[CLS]"] + test_tokens + ["[SEP]"]

In [0]:
# input_ids = get_ids(test_tokens, tokenizer, max_seq_length)
# input_masks = get_masks(test_tokens, max_seq_length)
# input_segments = get_segments(test_tokens, max_seq_length)

In [0]:
# pool_embs, all_embs = model.predict(
#      [[input_ids], [input_masks], [input_segments]]
# )

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [0]:
nizer = Tokenizer()

In [0]:
nizer.fit_on_texts(data['corpus'])

In [0]:
sparse = nizer.texts_to_matrix(data['corpus'])
print(sparse.shape)

In [0]:
sparse[0]

In [0]:
# data_strat0.to_csv(index=False)

In [0]:
# %%capture
# from tqdm import tqdm_notebook as tqdm
# tqdm().pandas()

In [0]:
# tokens = []

# for doc in tqdm(nlp.pipe(datasamp['corpus'], batch_size=500)):
    
#     doc_tokens = []
    
#     for token in doc:
#         if (token.is_stop == False) & (token.is_punct == False):
#             doc_tokens.append(token.lemma_.lower())
            
#     tokens.append(doc_tokens)
        
# datasamp['tokens'] = tokens

# del tokens

# datasamp.head()